# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

Zad 1

In [21]:
length=46 #wybierz długość
zad1="SELECT * FROM film WHERE length= " + str(length) + " ORDER BY title "
pd.read_sql(zad1,con=connection)


,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,15,Alien Center,A Brilliant Drama of a Cat And a Mad Scientist...,2006,1,5,2.99,46,10.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Behind the Scenes]",'alien':1 'battl':15 'brilliant':4 'cat':8 'ce...
1,469,Iron Moon,A Fast-Paced Documentary of a Mad Cow And a Bo...,2006,1,7,4.99,46,27.99,PG,2013-05-26 14:50:58.951,"[Commentaries, Behind the Scenes]",'baloon':22 'boy':14 'cow':11 'dentist':19 'do...
2,504,Kwai Homeward,A Amazing Drama of a Car And a Squirrel who mu...,2006,1,5,0.99,46,25.99,PG-13,2013-05-26 14:50:58.951,"[Trailers, Commentaries]","'amaz':4 'car':8,16 'drama':5 'georgia':19 'ho..."
3,505,Labyrinth League,A Awe-Inspiring Saga of a Composer And a Frisb...,2006,1,6,2.99,46,24.99,PG-13,2013-05-26 14:50:58.951,"[Commentaries, Behind the Scenes]",'awe':5 'awe-inspir':4 'compos':10 'desert':22...
4,730,Ridgemont Submarine,A Unbelieveable Drama of a Waitress And a Comp...,2006,1,3,0.99,46,28.99,PG-13,2013-05-26 14:50:58.951,"[Commentaries, Deleted Scenes, Behind the Scenes]",'ancient':19 'compos':11 'cow':17 'drama':5 'j...


Zad 2

In [6]:
#od client from city
city="York"
zad2 = "SELECT city.city, customer.first_name, customer.last_name FROM customer INNER JOIN address USING (address_id) INNER JOIN city USING (city_id) WHERE customer.address_id IN ( SELECT address.address_id FROM address WHERE city_id IN ( SELECT city.city_id FROM city WHERE city = ('"+ city + "') ) ) ORDER BY customer.first_name, customer.last_name ASC "
pd.read_sql(zad2,con=connection)

,city,first_name,last_name
0,York,Gilbert,Sledge


Zad 3

In [7]:
zad3="SELECT AVG(rental_rate) FROM film"
pd.read_sql(zad3,con=connection)

,avg
0,2.98


Zad 4

In [9]:
#od number_films_in_category z usuniętym warunkiem where
zad4 = "SELECT category.name AS category, COUNT(film.title) FROM film INNER JOIN film_category ON film.film_id = film_category.film_id INNER JOIN category USING (category_id) GROUP BY category.name "
pd.read_sql(zad4,con=connection)

,category,count
0,Sports,74
1,Classics,57
2,New,63
3,Family,69
4,Comedy,58
5,Animation,66
6,Travel,57
7,Music,51
8,Drama,62
9,Horror,56


Zad 5

In [11]:
#Client from city, z modyfikacją jak dodanym krajem
#zad5 = "SELECT country.country, city.city, customer.first_name, customer.last_name FROM customer INNER JOIN address USING (address_id) INNER JOIN city USING (city_id) INNER JOIN country USING (country_id) ORDER BY country.country, customer.first_name, customer.last_name ASC "
#zadanie wymaga liczby nie listy...
zad5 = "SELECT country.country, COUNT(country.country) FROM customer INNER JOIN address USING (address_id) INNER JOIN city USING (city_id) INNER JOIN country USING (country_id) GROUP BY country.country ORDER BY country.country ASC"
pd.read_sql(zad5,con=connection)

,country,count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
5,Argentina,13
6,Armenia,1
7,Austria,3
8,Azerbaijan,2
9,Bahrain,1


Zad 6

In [25]:
zad6 = "SELECT store.store_id, COUNT(customer.customer_id) FROM customer INNER JOIN store USING (store_id) GROUP BY store.store_id HAVING COUNT(customer.customer_id) > 100 AND COUNT(customer.customer_id) < 300"
pd.read_sql(zad6,con=connection)

,store_id,count
0,2,273


Zad 7

In [27]:
#Jeżeli jak długo oglądali filmy jest wzięte dosłownie (0 znalezionych wyników spełniających warunek)
#sum_min = 200*60
#zad7 ="SELECT customer.first_name, customer.last_name, SUM(film.length) FROM customer INNER JOIN rental USING (customer_id) INNER JOIN inventory USING (inventory_id) INNER JOIN film USING (film_id) GROUP BY customer.first_name, customer.last_name HAVING SUM(film.length) >= " + str(sum_min) + "ORDER BY SUM(film.length), customer.first_name, customer.last_name ASC "

#Jeżeli jak długo oglądali film oznacza czy kiedyś wypożyczali filmy przez dłużej niż 200 h...
zad7 ="SELECT customer.first_name, customer.last_name, MAX(rental.return_date-rental.rental_date) FROM customer INNER JOIN rental USING (customer_id) GROUP BY customer.first_name, customer.last_name HAVING MAX(rental.return_date-rental.rental_date) >= " + "('8 days 08:00:00')" + " ORDER BY customer.first_name, customer.last_name ASC "
pd.read_sql(zad7,con=connection)



,first_name,last_name,max
0,Aaron,Selby,9 days 00:36:00
1,Adam,Gooch,8 days 23:50:00
2,Adrian,Clary,8 days 20:35:00
3,Agnes,Bishop,9 days 05:19:00
4,Alan,Kahn,9 days 01:12:00
5,Albert,Crouse,9 days 04:50:00
6,Alex,Gresham,9 days 05:27:00
7,Alexander,Fennell,9 days 02:54:00
8,Alfred,Casillas,9 days 03:11:00
9,Alfredo,Mcadams,9 days 02:51:00


Zad 8

In [16]:
#lekka modyfikacja zad 3
zad8="SELECT AVG(amount) FROM payment"
pd.read_sql(zad8,con=connection)
#Wynik jest większy niż z zad 3, widocznie w bazie danych są częściej wypożyczene droższe filmy

,avg
0,4.200606


Zad 9

In [15]:
#lekka modyfikacja zad 4 AVG zamiast count
zad9 = "SELECT category.name AS category, AVG(film.length) FROM film INNER JOIN film_category ON film.film_id = film_category.film_id INNER JOIN category USING (category_id) GROUP BY category.name "
pd.read_sql(zad9,con=connection)

,category,avg
0,Sports,128.202703
1,Classics,111.666667
2,New,111.126984
3,Family,114.782609
4,Comedy,115.827586
5,Animation,111.015152
6,Travel,113.315789
7,Music,113.647059
8,Drama,120.838710
9,Horror,112.482143


Zad 10

In [20]:
#lekka modyfikacja zad 10 MAX LEN title zamiast count
zad10 = "SELECT category.name AS category, MAX(LENGTH(film.title)) FROM film INNER JOIN film_category ON film.film_id = film_category.film_id INNER JOIN category USING (category_id) GROUP BY category.name "
pd.read_sql(zad10,con=connection)


,category,max
0,Sports,25
1,Classics,23
2,New,21
3,Family,22
4,Comedy,23
5,Animation,22
6,Travel,23
7,Music,22
8,Drama,22
9,Horror,27


Zad 11

In [17]:
#lekka modyfikacja zad 4 MAX length zamiast count
zad11 = "SELECT category.name AS category, MAX(film.length) FROM film INNER JOIN film_category ON film.film_id = film_category.film_id INNER JOIN category USING (category_id) GROUP BY category.name "
pd.read_sql(zad11,con=connection)


,category,max
0,Sports,184
1,Classics,184
2,New,183
3,Family,184
4,Comedy,185
5,Animation,185
6,Travel,185
7,Music,185
8,Drama,181
9,Horror,181


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

Screenshot od testów z test_main znajduje się w tym samym folderze, co ten raport, poniższe przykłady pokazują, jak funkcje działają w notatniku po importowaniu main'a

In [1]:
import main

In [3]:
main.film_in_category(1)

,title,language,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
5,Berets Agent,English,Action
6,Bride Intrigue,English,Action
7,Bull Shawshank,English,Action
8,Caddyshack Jedi,English,Action
9,Campus Remember,English,Action


In [6]:
a=main.film_in_category(155) #ma być pusta odpowiedź
type(a)

NoneType

In [14]:
main.number_films_in_category(1)

,category,count
0,Action,64


In [15]:
main.number_films_in_category(155) #ma być pusta odpowiedź

In [16]:
main.number_film_by_length(46,56)

,length,count
0,46,5
1,47,7
2,48,11
3,49,5
4,50,9
5,51,7
6,52,7
7,53,9
8,54,6
9,55,2


In [17]:
main.number_film_by_length(45,45) #ma być pusta odpowiedź

In [19]:
main.client_from_city("Athenai")

,city,first_name,last_name
0,Athenai,Linda,Williams


In [20]:
main.client_from_city("Lethbridge") #ma być pusta odpowiedź

In [21]:
main.avg_amount_by_length(46)

,length,avg
0,46,3.638352


In [22]:
main.avg_amount_by_length(45) #ma być pusta odpowiedź

In [25]:
main.client_by_sum_length(1265)

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
5,Jerome,Kenyon,1679
6,Penny,Neal,1738
7,Lonnie,Tirado,1834
8,Dwight,Lombardi,1846
9,Johnny,Turpin,1846


In [24]:
main.client_by_sum_length(5389)#ma być pusta odpowiedź

In [26]:
main.category_statistic_length("Action")

,category,avg,sum,min,max
0,Action,111.609375,7143,47,185


In [28]:
main.category_statistic_length("Cartoon") #ma być pusta odpowiedź